Cleaning Data - Curso DATACAMP

In [ ]:
# Find duplicates
duplicates = ride_sharing.duplicated('ride_id', keep= False)

# Sort your duplicated rides
duplicated_rides = ride_sharing[duplicates].sort_values('ride_id')

# Print relevant columns of duplicated_rides
print(duplicated_rides[['ride_id', 'duration', 'user_birth_year']])

# Drop complete duplicates from ride_sharing
ride_dup = ride_sharing.drop_duplicates()

# Create statistics dictionary for aggregation function
statistics = {'user_birth_year': 'min', 'duration': 'mean'}

# Group by ride_id and compute new statistics
ride_unique = ride_dup.groupby('ride_id').agg(statistics).reset_index()

# Find duplicated values again
duplicates = ride_unique.duplicated(subset = 'ride_id', keep = False)
duplicated_rides = ride_unique[duplicates == True]

# Assert duplicates are processed
assert duplicated_rides.shape[0] == 0

Buscando dados inconsistentes

In [ ]:
# Find the cleanliness category in airlines not in categories
# buscando os dados que estão na coluna cleanlines do df airlines e que NÃO estão no domínio categories
# Aqui é mostrado apenas os dados
cat_clean = set(airlines['cleanliness']).difference(categories['cleanliness'])

# Find rows with that category
# define AS LINHAS onde os dados estão inconsistentes
cat_clean_rows = airlines['cleanliness'].isin(cat_clean)

# Print rows with inconsistent category
# Mostra as linhas completas onde há diferença entre o df e o categories (referência)
print(airlines[cat_clean_rows])

# Print rows with consistent categories only
# Mostra apenas as linhas com dados CONSISTENTES (ao contrário do consistente)
print(airlines[~cat_clean_rows])

Categorizar (Replace Mapping)

In [ ]:
# Create ranges for categories
label_ranges = [0, 60, 180, np.inf]
label_names = ['short', 'medium', 'long']

# Create wait_type column
airlines['wait_type'] = pd.cut(airlines['wait_min'], bins = label_ranges, 
                                labels = label_names)

# Create mappings and replace
mappings = {'Monday':'weekday', 'Tuesday':'weekday', 'Wednesday': 'weekday', 
            'Thursday': 'weekday', 'Friday': 'weekday', 
            'Saturday': 'weekend', 'Sunday': 'weekend'}

airlines['day_week'] = airlines['day'].replace(mappings)

Usando Regex e substituindo

In [ ]:
# Replace "-" with nothing
phones["Phone number"] = phones["Phone number"].str.replace("-", "")

# Replace phone numbers with lower than 10 digits to NaN
digits = phones['Phone number'].str.len()
phones.loc[digits < 10, "Phone number"] = np.nan

# Find length of each row in Phone number column
sanity_check = phone['Phone number'].str.len()
# Assert minmum phone number length is 10
assert sanity_check.min() >= 10
# Assert all numbers do not have "+" or "-"
assert phone['Phone number'].str.contains("+|-").any() == False

# Remember, assert returns nothing if the condition passes

# REGEX - Replace letters with nothing USANDO REGEX
phones['Phone number'] = phones['Phone number'].str.replace(r'\D+', '')

Uniformity - Conversão de dados (Datas e demais situações)

In [ ]:
# Find values of acct_cur that are equal to 'euro'
acct_eu = banking['acct_cur'] == 'euro'

# Convert acct_amount where it is in euro to dollars
# nesse caso ele busca o match acct_eu (já definido acima) na coluna 'acct_amount'
banking.loc[acct_eu, 'acct_amount'] = banking.loc[acct_eu, 'acct_amount'] * 1.1 

# Unify acct_cur column by changing 'euro' values to 'dollar'
banking.loc[acct_eu, 'acct_cur'] = 'dollar'

# Assert that only dollar currency remains
assert banking['acct_cur'].unique() == 'dollar'

# Print the header of account_opend
print(banking['account_opened'].head())

# Convert account_opened to datetime
# Força a conversão de datas quando não encontra uma data válida (mês maior que 12 ou dias maior que 31)
banking['account_opened'] = pd.to_datetime(banking['account_opened'],
                                          # Infer datetime format
                                          infer_datetime_format = True,
                                          # Return missing value for error
                                          errors = 'coerce') 

# Get year of account opened
# Converte a coluna em formato ano
banking['acct_year'] = banking['account_opened'].dt.strftime('%Y')

# Print acct_year
print(banking.acct_year)

Cross Validation - Uma forma de verificar consistência a partir de soma ou agrupamento de colunas.

In [ ]:
import pandas as pd
import datetime as dt

# Store fund columns to sum against
# Define as colunas para somar
fund_columns = ['fund_A', 'fund_B', 'fund_C', 'fund_D']

# Find rows where fund_columns row sum == inv_amount
# compara colunas somadas (usar axis = 1 para somar colunas) com outra coluna
inv_equ = banking[fund_columns].sum(axis = 1) == banking['inv_amount']

# Store consistent and inconsistent data
# o que for consistente e inconsistente separa
consistent_inv = banking[inv_equ]
inconsistent_inv = banking[~inv_equ]

# Store consistent and inconsistent data
print("Number of inconsistent investments: ", inconsistent_inv.shape[0])

# Mesmo exemplo com datas de aniversário
# Store today's date and find ages
# Store today's date and find ages
today = dt.date.today()
ages_manual = today.year - banking['birth_date'].dt.year

# Find rows where age column == ages_manual
age_equ = banking['age'] == ages_manual

# Store consistent and inconsistent data
consistent_ages = banking[age_equ]
inconsistent_ages = banking[~age_equ]

Missing Values

In [ ]:
# Analisando Valores Faltantes
#IMportar dados e analisar missing valores usando biblioetaca Missingno

import missingno as msno #biblioteca missingno - bem interessante
import matplotlib.pyplot as plt
import pandas as pd

# Print number of missing values in banking
print(banking.isna().sum())

# Visualize missingness matrix
msno.matrix(banking)
plt.show()

# Isolate missing and non missing values of inv_amount
missing_investors = banking[banking['inv_amount'].isna()]
investors = banking[~banking['inv_amount'].isna()]

# Sort banking by age and visualize
banking_sorted = banking.sort_values(by='age')
msno.matrix(banking_sorted)
plt.show()

In [ ]:
# Substituindo valores faltantes:
# Drop missing values of cust_id
banking_fullid = banking.dropna(subset = ['cust_id'])

# Compute estimated acct_amount
acct_imp = banking_fullid['inv_amount']*5

# Impute missing acct_amount with corresponding acct_imp
banking_imputed = banking_fullid.fillna({'acct_amount':acct_imp})

# Print number of missing values
print(banking_imputed.isna().sum())